In [1]:

!pip install nltk
!pip install xgboost
!pip install imblearn
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer



#embedding_2=pd.read_csv('../xwang3306/aliba_embedding_10p.csv')

In [2]:

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
import pandas as pd


data_size = '10p'
# data_size = 'full'
version = 'v0'

data_10P = pd.read_csv('NEISS_10P_Sample.csv') if data_size == '10p' else pd.read_csv('../data/consolidated_cleaned_neiss_2014_2023.csv')

# data_10P = pd.read_csv('../data/neiss_10p_sample.csv')
if data_size == '10p':
    data_10P = data_10P.drop(columns=['Unnamed: 0'])

print("data_10p", data_10P.shape)

embedding=pd.read_csv(f'gist_embedding_{data_size}_{version}.csv')
print("embedding", embedding.shape)
# print("new_columns", new_columns.shape)
print(embedding.head(10))

[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


data_10p (352052, 28)
embedding (346312, 385)
   CPSC_Case_Number         0         1         2         3         4  \
0         221032332 -0.041426 -0.019759  0.047550 -0.016496 -0.010686   
1         181109464 -0.017625 -0.034500  0.065491  0.020524  0.021781   
2         210103105 -0.056189 -0.016024  0.018050  0.011191 -0.066217   
3         161157997 -0.039718 -0.044306  0.027221  0.031631 -0.025631   
4         181107411 -0.030849 -0.014415  0.030544 -0.059319 -0.020998   
5         200134239  0.000362  0.013022  0.013661 -0.009646 -0.016427   
6         140951498 -0.044273  0.000546  0.037143 -0.014956 -0.030164   
7         221017396 -0.032238 -0.021063  0.026084 -0.042346 -0.011767   
8         200645623 -0.011380 -0.037507  0.041981 -0.064022 -0.016811   
9         141040420  0.019674 -0.015743  0.015878 -0.029989  0.005483   

          5         6         7         8  ...       374       375       376  \
0  0.015076  0.113167  0.052858 -0.000614  ...  0.034040 -0.036666  0.

In [3]:
# data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data_10P = data_10P[['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1', 'Disposition']]
data_2=data_10P.merge(embedding,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
del data_10P
del embedding
# data_10P.head(10)
# embedding.head(10)
#print(data.shape)
data=data_2.sample(frac=1)

In [4]:
print(list(data.columns))

['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1', 'Disposition', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', 

In [5]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [6]:
data=data[(data['Body_Part']!=0) & (data['Body_Part']!=84) & (data['Body_Part']!=85) & (data['Body_Part']!=86) & (data['Body_Part']!=87)]
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts()

0    296227
1     32874
Name: Disposition_recode_2, dtype: int64

In [7]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [8]:
data['body_string']=data['Body_Part'].map(bdpt_dict)
# data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [9]:

total_rows, n_columns = data.shape

test_size = int(total_rows * 0.2)
train_size = total_rows - test_size

print(f"Original dataset shape: ({total_rows}, {n_columns})")
print(f"Train set size: {train_size} rows")
print(f"Test set size: {test_size} rows")
data_sample = data.sample(frac=1,random_state=42).reset_index(drop=True)

data_ready = data_sample.tail(train_size).reset_index(drop=True)
data_ready_test = data_sample.head(test_size).reset_index(drop=True)
del data

# data_test=data_core.head(21000).reset_index(drop=True)
# data_fit=data_core.tail(191347).reset_index(drop=True)
# df_bad=data_fit[data_fit['Disposition_recode_2']==1]
# df_good=data_fit[data_fit['Disposition_recode_2']==0]
# data_good_sample=df_good.sample(frac=0.2,random_state=42).reset_index(drop=True)
# data_bad_sample=df_bad.sample(frac=0.8,random_state=42).reset_index(drop=True)
# data_core_sample=pd.concat([data_good_sample,data_bad_sample]).reset_index(drop=True)
# data_core_sample=data_core_sample.sample(frac=1,random_state=42).reset_index(drop=True)
# data_core_sample.head()

# data_ready = data_core_sample
# data_ready_test = data_test
data_ready.columns

Original dataset shape: (329101, 394)
Train set size: 263281 rows
Test set size: 65820 rows


Index(['CPSC_Case_Number', 'Age', 'Sex', 'Body_Part', 'Location', 'Product_1',
       'Disposition', '0', '1', '2',
       ...
       '377', '378', '379', '380', '381', '382', '383', 'Disposition_recode',
       'Disposition_recode_2', 'body_string'],
      dtype='object', length=394)

In [10]:
drop_list=[
 'CPSC_Case_Number',
 'Disposition',
 'Disposition_recode',
 'Disposition_recode_2',
 'body_string',
]

drop_list_test = drop_list

In [11]:
X = data_ready.drop(drop_list, axis=1)
y = data_ready['Disposition_recode_2']

print(list(X.columns))

['Age', 'Sex', 'Body_Part', 'Location', 'Product_1', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', 

In [12]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import numpy as np
from sklearn.neural_network import MLPClassifier
from xgboost import DMatrix, train

# Separate features (X) and target variable (y)
X = data_ready.drop(drop_list, axis=1)
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_ready_test.drop(drop_list_test, axis=1)
y_test=data_ready_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open(f'X_scaler_{data_size}_{version}.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Load your dataset (ensure X and y are defined properly)
# Assuming `X_train_final`, `X_test_final`, `y_train_final`, `y_test` are already defined

# Convert datasets to DMatrix format (required for xgb.train)
dtrain = DMatrix(data=X_train_final, label=y_train_final)
dtest = DMatrix(data=X_test_final, label=y_test)

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.1,
    'max_depth': 7,
    'scale_pos_weight': 1
}

model = train(params, dtrain, num_boost_round=5000, evals=[(dtest, 'eval')], verbose_eval=False)
y_prob = model.predict(dtest)
y_pred = (y_prob > 0.5).astype(int)
auc = roc_auc_score(y_test, y_prob)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"AUC: {auc}")
print(classification_report(y_test, y_pred))

with open(f'xgboost_embedding_Lower_v0.pkl', 'wb') as file:
    pickle.dump(model, file)

NameError: name 'xxxxx' is not defined

In [13]:
auc

0.8522477140213411

In [ ]:
X_test_final.head(5)


In [ ]:
scaled_body_parts = X_test_final['Body_Part'].unique()
scaled_body_parts.sort()
scaled_body_parts

In [ ]:
for index, row in X_test_final[:10].iterrows():
    prob = []
    for body_part in scaled_body_parts:
        current_sample = row.copy()
        current_sample['Body_Part'] = body_part
        # print(current_sample)
        prob.append(best_model.predict_proba([current_sample])[0][0])
    prob.sort()
    print(prob[0], prob[-1]) # min max

In [ ]:
%reset -f